In [1]:
from leibnizgym.envs.trifinger import TrifingerEnv
from isaacgym import gymtorch
import torch

Importing module 'gym_38' (/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.9.1+cu102
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...
WARNING - pyDR not installed. Domain Randomization functionality will not be available.


In [2]:
env_config = dict(num_instances=128,
     aggregate_mode=False,
     control_decimation=True,
     command_mode='torque',
     sim=dict(use_gpu_pipeline=True, physx=dict(use_gpu=False)))
env = TrifingerEnv(config=env_config, device='cuda:0', verbose=True, visualize=False)

[INFO] [2021.10.26::10-09-47]: [env_base]: Environment configuration: 
    seed: 0
    num_instances: 128
    spacing: 1.0
    control_decimation: True
    episode_length: 750
    aggregate_mode: False
    physics_engine: physx
    sim: 
        dt: 0.02
        substeps: 2
        up_axis: z
        gravity: [0.0, 0.0, -9.81]
        num_client_threads: 0
        use_gpu_pipeline: True
        physx: 
            solver_type: 1
            num_position_iterations: 4
            num_velocity_iterations: 0
            num_threads: 4
            use_gpu: False
            num_subscenes: 0
            max_gpu_contact_pairs: 8388608
        flex: 
            shape_collision_margin: 0.01
            num_outer_iterations: 4
            num_inner_iterations: 10
    pydr: 
        activate: False
        properties: 
            actions: 
                noise_operator: additive
                distribution: 
                    type: gaussian
                    stdev: [0.05, 0.05, 0.05, 0.0

In [7]:
Ji = env._gym.acquire_jacobian_tensor(env._sim, "robot")
Ji_t = gymtorch.wrap_tensor(Ji)

In [8]:
def compute_torque(ftip_force):
    env._gym.refresh_jacobian_tensors(env._sim)
    torque = 0
    for fid, frame_id in enumerate(env._fingertips_handles.values()):
        Ji = Ji_t[:, frame_id - 1, :3]
        Ji_T = Ji.transpose(1, 2)
        F = ftip_force[:, 3 * fid: 3 * fid + 3]
        torque += .5 * torch.matmul(Ji_T, F)
    return torque

In [23]:
def random_ftip_force():
    ac = torch.rand(env.get_action_shape(), dtype=torch.float, device=env.device)
    ac = ac.view(-1, 9, 1)
    return compute_torque(ac)

In [14]:
env._dof_state.shape

torch.Size([128, 9, 2])

In [17]:
action = 2 * torch.rand(env.get_action_shape(), dtype=torch.float, device=env.device) - 1

In [18]:
_, _, _, _ = env.step(action)

In [20]:
link_dict = env._gym.get_asset_rigid_body_dict(env._gym_assets['robot'])
dof_dict = env._gym.get_asset_dof_dict(env._gym_assets['robot'])

In [25]:
env._gym.refresh_jacobian_tensors(env._sim)

True

In [29]:
env._fingertips_handles

OrderedDict([('finger_tip_link_0', 6),
             ('finger_tip_link_120', 11),
             ('finger_tip_link_240', 16)])

In [24]:
link_dict

{'base_link': 0,
 'finger_base_link_0': 2,
 'finger_base_link_120': 7,
 'finger_base_link_240': 12,
 'finger_lower_link_0': 5,
 'finger_lower_link_120': 10,
 'finger_lower_link_240': 15,
 'finger_middle_link_0': 4,
 'finger_middle_link_120': 9,
 'finger_middle_link_240': 14,
 'finger_tip_link_0': 6,
 'finger_tip_link_120': 11,
 'finger_tip_link_240': 16,
 'finger_upper_link_0': 3,
 'finger_upper_link_120': 8,
 'finger_upper_link_240': 13,
 'upper_holder_link': 1}